In [ ]:
import seaborn as sns
df = sns.load_dataset('tips')

In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [ ]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [ ]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
encoder.fit_transform(df['time'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [ ]:
X = df.drop(labels=['time'],axis=1)

In [ ]:
y= df.time

In [ ]:
df['time']=encoder.fit_transform(df['time'])

In [ ]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [ ]:
X = df.drop(labels=['time'],axis=1)

In [ ]:
y= df.time

In [ ]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.20,random_state =42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


In [ ]:
cat_col = ["sex","smoker","day"]
num_col = ["total_bill","tip","size"]

In [ ]:
num_pipeline = Pipeline(
    steps= [
        ("imputer",SimpleImputer(strategy = "median")),
        ("scaler",StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy = "most_frequent")),
        ("scaler",OneHotEncoder())
    ]
)

In [ ]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,num_col),
    ('cat_pipeline',cat_pipeline,cat_col)
])

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
models = {
    "random_forest": RandomForestClassifier(oob_score=True),
    "logistic_regression": LogisticRegression(),
    "decision_tree": DecisionTreeClassifier()
}

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
  report={}
  for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    #print(model.score(X_train,y_train))
    #print("OOB score:",model.oob_score_)

    y_pred = model.predict(X_test)

    accuracy = round(accuracy_score(y_test,y_pred)*100,2)

    report[list(models.keys())[i]] = accuracy

  return report




In [ ]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'random_forest': 95.92, 'logistic_regression': 100.0, 'decision_tree': 93.88}

This increased no of accuracy is due to very small size of dataset

In [ ]:
params = {
    "n_estimators" : [50,100,200],
    "criterion": ["gini","entropy"],
    "max_depth": [3,5,10],

}

In [ ]:
model= RandomForestClassifier(oob_score=True)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Hyper parameter tuning with random forest

In [ ]:
cv=RandomizedSearchCV(model,param_distributions=params,scoring ='accuracy',cv=5,verbose=3)

In [ ]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.949 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.949 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(oob_score=True),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy', verbose=3)

In [ ]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'gini'}

In [ ]:
bestmodel=RandomForestClassifier(n_estimators=200, max_depth=10, criterion='gini',oob_score= True)

In [ ]:
bestmodel.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200, oob_score=True)

In [ ]:
y_pred = bestmodel.predict(X_test)
best_accuracy = round(accuracy_score(y_test,y_pred)*100,2)

In [ ]:
best_accuracy

95.92